In [5]:
import matplotlib
matplotlib.use("Agg")
 
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import cv2
import argparse
import random
import os

In [6]:
# initialize the data and labels
data = []
labels = []

In [7]:
# grab the image paths and randomly shuffle them
imagePaths = sorted(list(paths.list_images('images/')))
random.seed(42)
random.shuffle(imagePaths)

## Resize, Flatten and append to data,label lists
### Flattening: 32x32x3 = 3072

In [8]:
for imagePath in imagePaths:
	image = cv2.imread(imagePath)
    # flatten the image into 32x32x3=3072
	image = cv2.resize(image, (32, 32)).flatten()
	data.append(image)
 
	# extract the class label from the image path and update the labels list
	label = imagePath.split(os.path.sep)[-2]
	labels.append(label)

In [9]:
data[:1][0]

array([255, 255, 255, ...,  49,  83, 137], dtype=uint8)

## Scale the raw pixel intensities to the range [0, 1] from [0, 255]

In [10]:
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

## Train, Test split of 75, 25

In [11]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data,
	labels, test_size=0.25, random_state=42)

In [12]:
# convert the labels from integers to vectors (for 2-class, binary
# classification you should use Keras' to_categorical function
# instead as the scikit-learn's LabelBinarizer will not return a
# vector)
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [33]:
# define the 3072-1024-256-3 architecture using Keras
model = Sequential()
model.add(Dense(1024, input_shape=(3072,), activation="sigmoid"))
model.add(Dense(512, activation="sigmoid")) #Hidden 1
# model.add(Dense(256, activation="sigmoid")) #Hidden 2
# model.add(Dense(148, activation="sigmoid")) #Hidden 3
model.add(Dense(len(lb.classes_), activation="softmax")) #How many predictions do you want?

In [43]:
# initialize our initial learning rate and # of epochs to train for
INIT_LR = 0.01
EPOCHS = 100
 
# compile the model using SGD as our optimizer and categorical
# cross-entropy loss (you'll want to use binary_crossentropy
# for 2-class classification)
print("[INFO] training network...")
opt = SGD(lr=INIT_LR)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

[INFO] training network...


In [44]:
# train the neural network
H = model.fit(trainX, trainY, validation_data=(testX, testY),
	epochs=EPOCHS, batch_size=9)

Train on 84 samples, validate on 29 samples
Epoch 1/100
84/84 [==============================] - 1s 7ms/step - loss: 1.1071 - acc: 0.7024 - val_loss: 1.5474 - val_acc: 0.2414
Epoch 2/100
84/84 [==============================] - 0s 2ms/step - loss: 1.0986 - acc: 0.6310 - val_loss: 1.5479 - val_acc: 0.2414
Epoch 3/100
84/84 [==============================] - 0s 3ms/step - loss: 1.0884 - acc: 0.7024 - val_loss: 1.5546 - val_acc: 0.4138
Epoch 4/100
84/84 [==============================] - 0s 2ms/step - loss: 1.0960 - acc: 0.6667 - val_loss: 1.4598 - val_acc: 0.4138
Epoch 5/100
84/84 [==============================] - 0s 2ms/step - loss: 1.0806 - acc: 0.6786 - val_loss: 1.4369 - val_acc: 0.4138
Epoch 6/100
84/84 [==============================] - 0s 3ms/step - loss: 1.0843 - acc: 0.7143 - val_loss: 1.5310 - val_acc: 0.3448
Epoch 7/100
84/84 [==============================] - 0s 2ms/step - loss: 1.0854 - acc: 0.6190 - val_loss: 1.4419 - val_acc: 0.3103
Epoch 8/100
84/84 [====================

Epoch 63/100
84/84 [==============================] - 0s 2ms/step - loss: 0.8134 - acc: 0.8452 - val_loss: 1.5077 - val_acc: 0.4483
Epoch 64/100
84/84 [==============================] - 0s 2ms/step - loss: 0.8028 - acc: 0.8095 - val_loss: 1.4126 - val_acc: 0.3103
Epoch 65/100
84/84 [==============================] - 0s 3ms/step - loss: 0.8111 - acc: 0.7976 - val_loss: 1.4149 - val_acc: 0.3793
Epoch 66/100
84/84 [==============================] - 0s 3ms/step - loss: 0.8033 - acc: 0.8214 - val_loss: 1.4854 - val_acc: 0.3448
Epoch 67/100
84/84 [==============================] - 0s 3ms/step - loss: 0.8028 - acc: 0.8214 - val_loss: 1.5179 - val_acc: 0.5172
Epoch 68/100
84/84 [==============================] - 0s 2ms/step - loss: 0.8002 - acc: 0.8452 - val_loss: 1.4262 - val_acc: 0.3448
Epoch 69/100
84/84 [==============================] - 0s 2ms/step - loss: 0.7882 - acc: 0.8452 - val_loss: 1.4778 - val_acc: 0.4138
Epoch 70/100
84/84 [==============================] - 0s 2ms/step - loss: 0.

In [45]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=lb.classes_))
 
# plot the training loss and accuracy
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["acc"], label="train_acc")
plt.plot(N, H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy (Simple NN)")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig('stats.png')

[INFO] evaluating network...
                     precision    recall  f1-score   support

       akshay_kumar       0.00      0.00      0.00         4
       katrina_kaif       0.17      0.25      0.20         4
nawazuddin_siddiqui       0.29      0.22      0.25         9
      ranbir_kapoor       0.40      0.40      0.40         5
        zarine_khan       0.50      0.57      0.53         7

          micro avg       0.31      0.31      0.31        29
          macro avg       0.27      0.29      0.28        29
       weighted avg       0.30      0.31      0.30        29

